### 청년창업사관학교

In [3]:
import json
import re

from docx import Document
from docx.shared import Pt
from openai import OpenAI

from docx2pdf import convert

In [ ]:
client = OpenAI(api_key='YOUR_API_KEY')

In [6]:
# 공용 프롬프트

kosme_template = {f'''

1. 문제 인식  
1) 제품 및 서비스 개발동기
(자사가 개발(보유)하고 있는 제품 및 서비스에 대한 개발동기 등을 기재)

2) 제품 및 서비스의 목적(필요성)
(제품(서비스)을 구현하고자 하는 목적, 고객의 니즈를 혁신적으로 해결하기 위한 방안 등을 기재)

2. 실현 가능성
1) 제품 및 서비스의 개발 방안
(제품(서비스) 구현정도ㅡ 제작 소요기간 및 제작방법(자체 외주), 추진일정 등을 기재)

3. 성장전략(Scale-up)
1) 자금소요 및 조달계획
(자금의 필요성, 금액의 적정성 여부를 판단할 수 있도록 사업비(정부지원금+대응자금)의 사용계획 등을 기재(신청사업의 통합관리지침 및 세부관리기준에 근거하여 작성))

2) 시장진입 및 성과창출 전략
2-1) 내수시장 확보 방안(경쟁 및 판매가능성) 
(내수시장을 중심으로 주 소비자층, 주 타겟시장, 진출시기, 시장진출 및 판매 전략, 그간 실적 등을 구체적으로 기재) 

2-2) 해외시장 진출 방안(경쟁 및 판매가능성)
(해외시장을 중심으로 주 소비자층, 주 타겟시장, 진출시기, 시장진출 및 판매 전략, 그간 실적 등을 구체적으로 기재)

4. 팀 구성
1) 대표자 및 직원의 보유역량 및 기술보호 노력

2) 사회적 가치 실천계획
(양질의 일자리 창출을 위한 중소기업 성과공유제, 비공유직의 정규직화, 근로시간 단축 등 사회적 가치 실천계획을 기재)

3) 지역특화 아이디어 기반 사업추진계획
(지역에 특화된 산업 및 문화 등을 신기술 또는 최신 트렌드와 결합한 측면에서 기술)

'''
}

In [7]:
# 공용 프롬프트

kosme_pass = {f'''
1. 사업아이템과 관련된 사회적으로 발생하는 문제점을 제시합니다
2. 사회적 문제에 대한 근거를 제시해 주세요
3. 시장 동향(트렌드)과 성장 가능성을 제시합니다.
4. 사업 시작하게 된 동기와 문제인식, 문제 정의를 제시해 주세요,
5. 시장에서 발견한 고객(이용자)의 니즈를 파악해서 제시해 주세요.
6. 시장에서 발견한 고객(이용자)의 pain-point를 파악해서 제시해 주세요.
7. 제품(서비스)을 구현하고자 하는 목적, 고객의 니즈를 혁신적으로 해결하기 위한 방안을 제시해 주세요.    
8. 가능하다면 시장 확장 가능성에 대해서도 제시해 주세요.
              '''}

In [ ]:
# 1. 문제인식 항목 

prompt_1 = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
아래 항목에 대해 각각 15줄 이상 상세히 답변해주세요. 반드시 답변마다 별도의 문단을 생성하고, 간결함보다 완전성을 우선시합니다.
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 제품 및 서비스 개발동기와 서비스의 목적&필요성 부분만 작성해 주세요.
시장의 필요성과 법제화가 중요합니다.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.
'특히' 또는 '결론적으로' 같은 단어가 있다면 줄바꿈을 해주세요.

문제 인식 1: {item_info['제품 및 서비스 개발동기']}
문제 인식 2: {item_info['제품 및 서비스의 목적_필요성']} 

**반드시 지켜야 할 출력 규칙**
1. 절대 #, *, - 기호 및 마크다운 스타일 절대 금지
2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함
4. 문장이 끝나면 띄어쓰기를 할것

예시 출력:  
1. 문제인식
   1) 제품 및 서비스 개발동기  
      본 제품은 시장의 OOO한 문제를 해결하기 위해 개발되었습니다...  

   2) 제품 및 서비스의 목적 및 필요성  
      현재 시장에서 유사 제품은 OOO한 한계를 가지고 있으며...  
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt_1}],
                                                            temperature=0.7,
                                                            presence_penalty=0.8,
                                                            frequency_penalty=0.5,
                                                            max_tokens = 4000)
                                                            
# presence_penalty: 이 값을 0.5~1.0 정도로 설정하면 모델이 새로운 주제나 단어를 도입하는 경향이 증가하여 출력의 다양성이 높아집니다.
# frequency_penalty: 이 값을 0.5~1.0 정도로 설정하면 특정 단어의 반복 사용을 억제하여 출력의 품질을 향상시킬 수 있습니다.

# 생성된 사업계획서 초안 출력
사업계획서_초안1 = completion.choices[0].message.content
사업계획서_초안1 = 사업계획서_초안1.replace("### ", "")  # 자동으로 붙은 '### '을 제거  
print("사업계획서 초안 :\n", 사업계획서_초안1)

사업계획서 초안 :
 1. 문제 인식
   1) 제품 및 서비스 개발동기  
      본 제품은 나무의사의 진료 기록을 작성하고 보관하는 체계가 부족하여 이를 해결하기 위해 개발되었습니다. 현재 나무병원에서는 수목의료를 위해 필요한 농약, 비료, 자재 등을 관리하는 시스템이 전무한 실정입니다. 이러한 문제로 인해 효율적인 수목관리와 응급상황 대처에 어려움이 있습니다. 특히, 진료 기록과 처방 약품 발주 및 관리를 위한 체계적인 시스템의 부재는 종종 정보 누락이나 오류를 발생시켜 결과적으로 수목 건강에 악영향을 미칠 수 있습니다. 이와 같은 산업 내의 비효율성을 개선하고자 OCS 기반의 수목 의료용 케미컬 및 자재 커머스 플랫폼을 개발하게 되었습니다.

   2) 제품 및 서비스의 목적 및 필요성  
      현재 시장에서 유사 제품은 대부분 특정 기능만 수행하거나 제한된 범위 내에서 사용되고 있어 포괄적인 솔루션 제공에 한계를 가지고 있습니다. 예를 들어, 일부 시스템은 단순한 재고 관리에 그치거나 특정 품목 발주에 국한되는 경우가 많습니다. 그러나 우리의 플랫폼은 나무병원의 모든 진료 과정과 관련 정보를 통합하고자 하는 목표를 가지고 있습니다. 이를 통해 나무병원뿐만 아니라 조경업체, 케미컬 회사, 조경담당자가 보다 효율적이고 정확하게 정보를 공유할 수 있도록 돕습니다.

특히

현재 한국 조경시장의 규모는 약 14조 원이며, 나무병원 시장만 해도 1000억 원으로 평가됩니다 (출처: 한국조경신문). 이러한 경제적 규모에도 불구하고 디지털화된 관리 시스템이 부재하여 많은 비용과 시간이 불필요하게 소모되고 있는 상황입니다.

결론적으로

본 플랫폼은 이러한 시장의 필요성을 충족시키고 법제화 지원 가능성을 염두에 두어 향후 표준화된 관리 시스템으로 자리잡아갈 것입니다. 이는 궁극적으로 산업 전체의 생산성과 효율성을 높이고 지속 가능한 환경 구축에 기여할 것입니다.


In [ ]:
# 2. 실현 가능성 항목 

prompt_2 = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
아래 항목에 대해 각각 15줄 이상 상세히 답변해주세요. 반드시 답변마다 별도의 문단을 생성하고, 간결함보다 완전성을 우선시합니다
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 제품 및 서비스의 개발 방안 부분만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.
'특히' 또는 '결론적으로' 같은 단어가 있다면 줄바꿈을 해주세요.

실현 가능성 1: {item_info['제품 및 서비스의 개발 방안']}

**반드시 지켜야 할 출력 규칙**
1. 절대 #, *, - 기호 및 마크다운 스타일 절대 금지
2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함
4. 문장이 끝나면 띄어쓰기를 할것

예시 출력: 
2. 실현 가능성
   1) 제품 및 서비스의 개발 방안 :
      본 제품은 시장의 OOO한 문제를 해결하기 위해 개발되었습니다...  

 
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt_2}],
                                                            temperature=0.7,
                                                            presence_penalty=0.8,
                                                            frequency_penalty=0.5,
                                                            max_tokens = 4000)
                                                            
# presence_penalty: 이 값을 0.5~1.0 정도로 설정하면 모델이 새로운 주제나 단어를 도입하는 경향이 증가하여 출력의 다양성이 높아집니다.
# frequency_penalty: 이 값을 0.5~1.0 정도로 설정하면 특정 단어의 반복 사용을 억제하여 출력의 품질을 향상시킬 수 있습니다.

# 생성된 사업계획서 초안 출력
사업계획서_초안2 = completion.choices[0].message.content
사업계획서_초안2 = 사업계획서_초안2.replace("### ", "")  # 자동으로 붙은 '### '을 제거  
print("사업계획서 초안 :\n", 사업계획서_초안2)

사업계획서 초안 :
 2. 실현 가능성  
   1) 제품 및 서비스의 개발 방안 :  
      본 제품은 시장의 수목관리 산업에서 발생하는 비효율성을 해결하기 위해 개발되었습니다. 현재 나무의사용 전자 의무기록(EMR) 시스템은 헬스케어 전문기업 시야인사이트와 한양대학교 공과대학원과 공동으로 개발이 완료된 상태입니다. 이 시스템은 나무병원 및 조경업체들이 수목 진료 기록을 디지털화하여 보관하고, 이를 통해 보다 효율적으로 수목 관리 작업을 수행할 수 있도록 설계되었습니다. 

      다음 단계로는 처방 전달 시스템인 OCS(Order Communication System)를 개발할 예정입니다. 이 시스템은 나무의사가 진단한 정보를 바탕으로 필요한 약품과 자재를 자동으로 발주하거나 추천해주는 기능을 포함합니다. 이를 통해 기존에 수작업으로 진행되던 재고 관리 및 발주 과정에서 발생하는 오류를 최소화하고 효율성을 극대화할 것입니다.

      특히, EMR 시스템의 데이터베이스와 OCS 플랫폼 간의 통합이 중요한데, 이를 위해 API(Application Programming Interface)를 활용하여 두 시스템 간 원활한 데이터 교환이 이루어질 수 있도록 할 계획입니다. 이를 통해 사용자 경험을 개선하고, 실시간 정보 공유가 가능한 환경을 구축하려 합니다.

      개발 일정은 EMR 시스템의 피드백을 반영하여 6개월 내로 OCS 시스템을 론칭하는 것을 목표로 하고 있으며, 외부 전문 업체와 협력하여 백엔드 기술 일부를 아웃소싱할 계획입니다. 또한, 내부 인력 채용도 병행하여 프로젝트 진행 속도를 높일 것입니다.

      결론적으로, 이러한 접근 방식은 궁극적으로 사용자 편리성을 증대시키고 운영 비용 절감을 도모함으로써 전체 수목관리 프로세스를 혁신할 것입니다. 이를 기반으로 초기 고객 확보 후 안정적 운영 체계를 마련하면 추가적인 기능 확장 및 타 지역으로의 사업 확장이 용이해질 것으로 기대됩니다.

      데이터 출처: 한국산림복지진

In [ ]:
# 3. 성장 전략 항목 

prompt_3 = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
아래 항목에 대해 각각 15줄 이상 상세히 답변해주세요. 반드시 답변마다 별도의 문단을 생성하고, 간결함보다 완전성을 우선시합니다
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 시장진입 및 성과창출 전략에서 내수시장 확보 방안(경쟁 및 판매가능성)과 해외시장 진출 방안(경쟁 및 판매가능성), 자금소요 및 조달계획 부분만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.
'특히' 또는 '결론적으로' 같은 단어가 있다면 줄바꿈을 해주세요.

성장 전략 1: {item_info['내수시장 확보 방안 경쟁 및 판매가능성']}
성장 전략 2: {item_info['해외시장 진출 방안 경쟁 및 판매가능성']}
성장 전략 3: {item_info['자금 조달 및 개발 방안']}

**반드시 지켜야 할 출력 규칙**
1. 절대 #, *, - 기호 및 마크다운 스타일 절대 금지
2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함
4. 문장이 끝나면 띄어쓰기를 할것

예시 출력:
3. 성장 전략  
   1) 내수시장 확보 방안 경쟁 및 판매가능성 :
      본 제품은 시장의 OOO한 문제를 해결하기 위해 개발되었습니다...  
   <주소비자 층 및 판매전략 표>
   예시) 표로 작성해주세요.("예시"라는 문구를 보여주세요.)


   2) 해외시장 진출 방안 경쟁 및 판매가능성 :
      현재 시장에서 유사 제품은 OOO한 한계를 가지고 있으며...
      
   3) 자금 조달 및 개발 방안 :
      현재 시장에서 유사 제품은 OOO한 한계를 가지고 있으며...
      
    <사업추진 일정(협약기간 내)>
    예시) 표로 작성해주세요.
    |구분|추진 내용|추진 기간|세부 내용|

    <사업비 집행 계획>
    *"정부지원사업비는 총 사업비의 70%이하, 현금은 10%이상, 현물은 20% 이하로 작성하세요.(최대 1억원 한도 이내)"라고 적어주세요.
    - 정부지원사업비가 총 사업비의 규정을 초과하지 않도록 주의하세요.
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt_3}],
                                                            temperature=0.7,
                                                            presence_penalty=0.8,
                                                            frequency_penalty=0.5,
                                                            max_tokens = 4000)
                                                            
# presence_penalty: 이 값을 0.5~1.0 정도로 설정하면 모델이 새로운 주제나 단어를 도입하는 경향이 증가하여 출력의 다양성이 높아집니다.
# frequency_penalty: 이 값을 0.5~1.0 정도로 설정하면 특정 단어의 반복 사용을 억제하여 출력의 품질을 향상시킬 수 있습니다.

# 생성된 사업계획서 초안 출력
사업계획서_초안3 = completion.choices[0].message.content
사업계획서_초안3 = 사업계획서_초안3.replace("### ", "")  # 자동으로 붙은 '### '을 제거  
print("사업계획서 초안 :\n", 사업계획서_초안3)

ValueError: Invalid format specifier ''system','content': prompt_3' for object of type 'str'

In [ ]:
# 4. 팀 구성 항목 

prompt_4 = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
아래 항목에 대해 각각 15줄 이상 상세히 답변해주세요. 반드시 답변마다 별도의 문단을 생성하고, 간결함보다 완전성을 우선시합니다
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 대표자 및 직원의 보유역량 및 기술보호 노력, 사회적 가치 실천계획 부분, 지역특화 아이디어 기반 사업추진계획만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.
'특히' 또는 '결론적으로' 같은 단어가 있다면 줄바꿈을 해주세요.

팀 구성 1: {item_info['대표자 및 직원의 보유역량 및 기술보호 노력']}
팀 구성 2: {item_info['사회적 가치 실천계획']}
팀 구성 3: {item_info['지역특화 아이디어 기반 사업추진계획']}

**반드시 지켜야 할 출력 규칙**
1. 절대 #, *, - 기호 및 마크다운 스타일 절대 금지
2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함
4. 문장이 끝나면 띄어쓰기를 할것

예시 출력: 
4. 팀 구성
1) 대표자 및 직원의 보유역량 및 기술보호 노력 :
   본 제품은 시장의 OOO한 문제를 해결하기 위해 개발되었습니다...  

2) 사회적 가치 실천계획 :
   현재 시장에서 유사 제품은 OOO한 한계를 가지고 있으며... 
    
3) 지역특화 아이디어 기반 사업추진계획 : 
   현재 시장에서 유사 제품은 OOO한 한계를 가지고 있으며...  
   
<팀 구성 표>
예시) 표로 작성해주세요.("예시"라는 문구를 보여주세요.)
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt_4}],
                                                            temperature=0.7,
                                                            presence_penalty=0.8,
                                                            frequency_penalty=0.5,
                                                            max_tokens = 4000)
                                                            
# presence_penalty: 이 값을 0.5~1.0 정도로 설정하면 모델이 새로운 주제나 단어를 도입하는 경향이 증가하여 출력의 다양성이 높아집니다.
# frequency_penalty: 이 값을 0.5~1.0 정도로 설정하면 특정 단어의 반복 사용을 억제하여 출력의 품질을 향상시킬 수 있습니다.

# 생성된 사업계획서 초안 출력
사업계획서_초안4 = completion.choices[0].message.content
사업계획서_초안4 = 사업계획서_초안4.replace("### ", "")  # 자동으로 붙은 '### '을 제거  
print("사업계획서 초안 :\n", 사업계획서_초안4)

In [ ]:
ccs_dict = {'초안1': 사업계획서_초안1,
            '초안2': 사업계획서_초안2,
            '초안3': 사업계획서_초안3,
            '초안4': 사업계획서_초안4}

In [ ]:
# 딕셔너리 데이터를 JSON 파일로 저장
with open('business_plan/청년창업사관학교_사업계획서.json', 'w', encoding='utf-8') as file:
    json.dump(ccs_dict, file, ensure_ascii=False, indent=4)

In [ ]:
# 예비창업패키지
# 초기창업패키지

# 문서 생성
doc = Document()

# 정규식을 사용하여 각 초안을 띄어쓰기 기준으로 나눔
for _, value in ccs_dict.items():
    re_text = re.split(r'\n\n', value)

    for idx in re_text:

        if '|' in idx:
            text = idx.split('|')
            doc.add_paragraph(text[0])

            data = text[1:-1]

            # '---'가 포함된 항목 제거 및 공백 정리
            filtered_data = [item.strip() for item in data if '-' not in item and item]

            # 데이터를 표 형식(2D 리스트)으로 변환
            table_data = []
            row = []

            for item in filtered_data:
                if item == "":  # 빈 문자열(\n 변환)일 경우 새로운 행 시작
                    if row:  # 현재까지 저장된 행이 있다면 추가
                        table_data.append(row)
                        row = []
                else:
                    row.append(item)

            # 마지막 행 추가 (루프 종료 후 남은 데이터 처리)
            if row:
                table_data.append(row)

            # 첫 번째 행을 자동으로 컬럼명으로 설정
            if table_data:
                headers = table_data.pop(0)  # 첫 번째 행을 컬럼명으로 설정
                table_data.insert(0, headers)  # 다시 삽입 (명확하게 구분하기 위함)


            # 표 생성 (행 개수는 데이터 행 개수, 열 개수는 첫 번째 행의 열 개수 기준)
            table = doc.add_table(rows=len(table_data), cols=len(table_data[0]))

            # 표 스타일 적용 (선택 사항: 표 테두리 추가)
            table.style = "Table Grid"

            # 표 데이터 삽입
            for row_idx, row_data in enumerate(table_data):
                for col_idx, cell_data in enumerate(row_data):
                    cell = table.cell(row_idx, col_idx)
                    cell.text = str(cell_data)  # 데이터 삽입
                    
                    # 첫 번째 행(컬럼명)은 굵게 표시
                    if row_idx == 0:
                        for paragraph in cell.paragraphs:
                            for run in paragraph.runs:
                                run.bold = True  # 글씨 굵게
                                run.font.size = Pt(12)  # 글씨 크기 조정 (선택 사항)

            doc.add_paragraph(text[-1])

        else:
            doc.add_paragraph(idx)

    doc.add_paragraph('\n\n') # 나눴던 구분자를 다시 집어넣음
    
# 파일 저장
doc.save("business_plan/청년창업사관학교_초안.docx")

print("Word 파일이 성공적으로 생성되었습니다!")

In [ ]:
convert("business_plan/청년창업사관학교_초안.docx", "business_plan/청년창업사관학교_초안.pdf")